# Inference

In [ ]:
"""
 @Time    : 9/29/19 17:14
 @Author  : TaylorMei
 @Email   : mhy666@mail.dlut.edu.cn
 
 @Project : ICCV2019_MirrorNet
 @File    : infer.py
 @Function: predict mirror map.
 
"""
import numpy as np
import os
import time

import torch
from PIL import Image
from torch.autograd import Variable
from torchvision import transforms

from config import msd_testing_root
from misc import check_mkdir, crf_refine
from mirrornet import MirrorNet

device_ids = [0]
torch.cuda.set_device(device_ids[0])

args = {"snapshot": "160", "scale": 384, "crf": True}
pretrain_directory = "/home/research/Datasets/NormalNet/dataset"
data_root = "/home/research/Datasets/NormalNet/MSD/test"
ckpt_path = "./final_results"
exp_name = "MirrorNet"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
batch_size = 30
learning_rate = 0.001
momentum = 0.9
power = 0.9
DIM = 384
pretrained_path = os.path.join(
    ckpt_path, f"{exp_name}_frozen_backbone_pretrain_e100.pth"
)
f1p = os.path.join(
    ckpt_path, f"{exp_name}_finetuned_e50_frozen_backbone_pretrain_e100.pth"
)
f2p = os.path.join(
    ckpt_path, f"{exp_name}_finetuned_e100_frozen_backbone_pretrain_e100.pth"
)
f3p = os.path.join(
    ckpt_path, f"{exp_name}_finetuned_e200_frozen_backbone_pretrain_e100.pth"
)

base_y = os.path.join(ckpt_path, f"{exp_name}_base.pth")
base = os.path.join(ckpt_path, f"{exp_name}_BASE.pth")
basev2 = os.path.join(ckpt_path, f"{exp_name}_BASE_V2.pth")
low = os.path.join(ckpt_path, f"{exp_name}_BASE_bs10_frozen.pth")
low2 = os.path.join(ckpt_path, f"{exp_name}_e200_frozen.pth")
# model_paths = [pretrained_path, f1p, f2p, f3p, base, basev2, low]
model_paths = [base_y, base, low, pretrained_path,f1p, f2p, f3p, low2]
labels = ["Yang et al. Mirrornet", "Repr.", "Repr. freeze", "Pretrain","FT 50E", "FT 100E", "FT 200E" ,"FT 50E Freeze"]


img_transform = transforms.Compose([
    transforms.Resize((args['scale'], args['scale'])),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

to_test = {'MSD': msd_testing_root}

to_pil = transforms.ToPILImage()


def main():
    for i, model_path in enumerate(model_paths):
            
        net = MirrorNet().cuda(device)
        net.load_state_dict(torch.load(model_path, map_location=device))
        net.eval()
        
        model_name = os.path.split(model_path)[1].split('.')[0]
        print(model_name)
        
        with torch.no_grad():
            for name, root in to_test.items():
                img_list = [
                    img_name
                    for img_name in os.listdir(
                        "/home/research/Datasets/NormalNet/MSD/test/image"
                        )
                ]                
                start = time.time()
                for idx, img_name in enumerate(img_list):
                    print('predicting for {}: {:>4d} / {}'.format(name, idx + 1, len(img_list)))
                    check_mkdir(os.path.join(ckpt_path, model_name))
                    img = Image.open(os.path.join("/home/research/Datasets/NormalNet/MSD/test/", 'image', img_name))
                    if img.mode != 'RGB':
                        img = img.convert('RGB')
                        print("{} is a gray image.".format(name))
                    w, h = img.size
                    img_var = Variable(img_transform(img).unsqueeze(0)).cuda(device_ids[0])
                    f_4, f_3, f_2, f_1 = net(img_var)
                    f_4 = f_4.data.squeeze(0).cpu()
                    f_3 = f_3.data.squeeze(0).cpu()
                    f_2 = f_2.data.squeeze(0).cpu()
                    f_1 = f_1.data.squeeze(0).cpu()
                    f_4 = np.array(transforms.Resize((h, w))(to_pil(f_4)))
                    f_3 = np.array(transforms.Resize((h, w))(to_pil(f_3)))
                    f_2 = np.array(transforms.Resize((h, w))(to_pil(f_2)))
                    f_1 = np.array(transforms.Resize((h, w))(to_pil(f_1)))
                    if args['crf']:
                        f_1 = crf_refine(np.array(img.convert('RGB')), f_1)

                    Image.fromarray(f_1).save(os.path.join(ckpt_path, model_name, img_name[:-4] + ".png"))

                end = time.time()
                print("Average Time Is : {:.2f}".format((end - start) / len(img_list)))


if __name__ == '__main__':
    main()

# Evaluations

In [1]:
from misc import *

pretrain_directory = "/home/research/Datasets/NormalNet/dataset"
data_root = "/home/research/Datasets/NormalNet/MSD/test"
ckpt_path = "./final_results"
exp_name = "MirrorNet"

pretrained_path = os.path.join(
    ckpt_path, f"{exp_name}_frozen_backbone_pretrain_e100"
)
f1p = os.path.join(
    ckpt_path, f"{exp_name}_finetuned_e50_frozen_backbone_pretrain_e100"
)
f2p = os.path.join(
    ckpt_path, f"{exp_name}_finetuned_e100_frozen_backbone_pretrain_e100"
)
f3p = os.path.join(
    ckpt_path, f"{exp_name}_finetuned_e200_frozen_backbone_pretrain_e100"
)

base_y = os.path.join(ckpt_path, f"{exp_name}_base")
base = os.path.join(ckpt_path, f"{exp_name}_BASE")
basev2 = os.path.join(ckpt_path, f"{exp_name}_BASE_V2")
low = os.path.join(ckpt_path, f"{exp_name}_BASE_bs10_frozen")
low2 = os.path.join(ckpt_path, f"{exp_name}_e200_frozen")
# model_paths = [pretrained_path, f1p, f2p, f3p, base, basev2, low]
model_paths = [base_y, base, low, pretrained_path,f1p, f2p, f3p, low2]
labels = ["Yang et al. Mirrornet", "Repr.", "Repr. freeze", "Pretrain","FT 50E", "FT 100E", "FT 200E" ,"FT 50E Freeze"]

result_dirs = [d.path for d in os.scandir(ckpt_path) if d.is_dir()]

# print(model_paths)

eval_results = {}
for i, model_out_path in enumerate(model_paths):
    images = os.listdir(model_out_path) # 955 output masks

    ious = []
    maes = []
    bers = []
    img_accs = []
    mir_accs = []
    f1s = []
    
    bin_ious = []
    bin_maes = []
    bin_bers = []
    bin_img_accs = []
    bin_mir_accs = []
    bin_f1s = []

    for image in images:
        pred_mask = get_normalized_predict_mask(image, model_out_path)
        bin_pred_mask = get_binary_predict_mask(image, model_out_path)
        gt_mask = get_gt_mask(image, "/home/research/Datasets/NormalNet/MSD/test/mask")
        iou = compute_iou(pred_mask, gt_mask)
        mae = compute_mae(pred_mask, gt_mask)
        ber = compute_ber(pred_mask, gt_mask)
        img_acc = compute_acc_image(pred_mask, gt_mask)
        mir_acc = compute_acc_mirror(pred_mask, gt_mask)
        f1 = compute_f1_score(pred_mask, gt_mask)

        ious.append(iou)
        maes.append(mae)
        bers.append(ber)
        img_accs.append(img_acc)
        mir_accs.append(mir_acc)
        f1s.append(f1)

        bin_iou = compute_iou(bin_pred_mask, gt_mask)
        bin_mae = compute_mae(bin_pred_mask, gt_mask)
        bin_ber = compute_ber(bin_pred_mask, gt_mask)
        bin_img_acc = compute_acc_image(bin_pred_mask, gt_mask)
        bin_mir_acc = compute_acc_mirror(bin_pred_mask, gt_mask)
        bin_f1 = compute_f1_score(bin_pred_mask, gt_mask)

        bin_ious.append(bin_iou)
        bin_maes.append(bin_mae)
        bin_bers.append(bin_ber)
        bin_img_accs.append(img_acc)
        bin_mir_accs.append(mir_acc)
        bin_f1s.append(f1)
    
    eval_results[labels[i]] = {
        "iou": np.mean(ious),
        "mae": np.mean(maes),
        "ber": np.mean(ber),
        "img_acc": np.mean(img_accs),
        "mir_acc": np.mean(mir_accs),
        "F1": np.mean(f1s),
        "bin_iou": np.mean(bin_ious),
        "bin_mae": np.mean(bin_maes),
        "bin_ber": np.mean(bin_ber),
        "bin_img_acc": np.mean(bin_img_accs),
        "bin_mir_acc": np.mean(bin_mir_accs),
        "bin_F1": np.mean(bin_f1s),
    }

    

In [2]:
print(eval_results)

{'Yang et al. Mirrornet': {'iou': 0.809494957887785, 'mae': 0.0652146555024994, 'ber': 0.01008167841406038, 'img_acc': 0.9296540009663368, 'mir_acc': 0.9440741500384885, 'F1': 0.8277720652938335, 'bin_iou': 0.7895902592385973, 'bin_mae': 0.06448828246156778, 'bin_ber': 0.01092632306739949, 'bin_img_acc': 0.9296540009663368, 'bin_mir_acc': 0.9440741500384885, 'bin_F1': 0.8277720652938335}, 'Repr.': {'iou': 0.2768506008614985, 'mae': 0.6128352494608045, 'ber': 0.4082482785615955, 'img_acc': 0.3207014153765134, 'mir_acc': 0.9875705141242884, 'F1': 0.6978553887046937, 'bin_iou': 0.2701785141701045, 'bin_mae': 0.6155266948317358, 'bin_ber': 0.34871046060906297, 'bin_img_acc': 0.3207014153765134, 'bin_mir_acc': 0.9875705141242884, 'bin_F1': 0.6978553887046937}, 'Repr. freeze': {'iou': 0.44536886073156273, 'mae': 0.3635899322624294, 'ber': 0.49991866658596373, 'img_acc': 0.28155098640481835, 'mir_acc': 0.9999506855138431, 'F1': 0.7438937443379275, 'bin_iou': 0.3848940069412915, 'bin_mae': 0.3

In [ ]:

{
    "Yang et al. Mirrornet": {
        "iou": 0.809494957887785,
        "mae": 0.0652146555024994,
        "ber": 0.01008167841406038,
        "img_acc": 0.9296540009663368,
        "mir_acc": 0.9440741500384885,
        "F1": 0.8277720652938335,
        "bin_iou": 0.7895902592385973,
        "bin_mae": 0.06448828246156778,
        "bin_ber": 0.01092632306739949,
        "bin_img_acc": 0.9296540009663368,
        "bin_mir_acc": 0.9440741500384885,
        "bin_F1": 0.8277720652938335,
    },
    "Repr.": {
        "iou": 0.2768506008614985,
        "mae": 0.6128352494608045,
        "ber": 0.4082482785615955,
        "img_acc": 0.3207014153765134,
        "mir_acc": 0.9875705141242884,
        "F1": 0.6978553887046937,
        "bin_iou": 0.2701785141701045,
        "bin_mae": 0.6155266948317358,
        "bin_ber": 0.34871046060906297,
        "bin_img_acc": 0.3207014153765134,
        "bin_mir_acc": 0.9875705141242884,
        "bin_F1": 0.6978553887046937,
    },
    "Repr. freeze": {
        "iou": 0.44536886073156273,
        "mae": 0.3635899322624294,
        "ber": 0.49991866658596373,
        "img_acc": 0.28155098640481835,
        "mir_acc": 0.9999506855138431,
        "F1": 0.7438937443379275,
        "bin_iou": 0.3848940069412915,
        "bin_mae": 0.3499061594517287,
        "bin_ber": 0.20665307326816573,
        "bin_img_acc": 0.28155098640481835,
        "bin_mir_acc": 0.9999506855138431,
        "bin_F1": 0.7438937443379275,
    },
    "Pretrain": {
        "iou": 0.5437651480787036,
        "mae": 0.4905460142964468,
        "ber": 0.4999735193535696,
        "img_acc": 0.22801787531188647,
        "mir_acc": 0.9999893705365991,
        "F1": 0.08076673941876311,
        "bin_iou": 0.03659219402805593,
        "bin_mae": 0.339263101616455,
        "bin_ber": 0.6446316163786581,
        "bin_img_acc": 0.22801787531188647,
        "bin_mir_acc": 0.9999893705365991,
        "bin_F1": 0.08076673941876311,
    },
    "FT 50E": {
        "iou": 1.0675482964536245,
        "mae": 0.10302615786039548,
        "ber": 0.37760456072390525,
        "img_acc": 0.5559545802570762,
        "mir_acc": 0.9769784504628152,
        "F1": 0.8083796236645495,
        "bin_iou": 0.681400955626929,
        "bin_mae": 0.08195863733072985,
        "bin_ber": 0.041460607491860424,
        "bin_img_acc": 0.5559545802570762,
        "bin_mir_acc": 0.9769784504628152,
        "bin_F1": 0.8083796236645495,
    },
    "FT 100E": {
        "iou": 1.2217599550399407,
        "mae": 0.12675240897812456,
        "ber": 0.49496489422873224,
        "img_acc": 0.2659121975723986,
        "mir_acc": 0.9976848253824844,
        "F1": 0.8847056028430149,
        "bin_iou": 0.6640654790070786,
        "bin_mae": 0.08561121749555137,
        "bin_ber": 0.019303877345762666,
        "bin_img_acc": 0.2659121975723986,
        "bin_mir_acc": 0.9976848253824844,
        "bin_F1": 0.8847056028430149,
    },
    "FT 200E": {
        "iou": 1.491340469318327,
        "mae": 0.12188642960217298,
        "ber": 0.07194815502557406,
        "img_acc": 0.8012103271484374,
        "mir_acc": 0.904825665811351,
        "F1": 0.4584271420251612,
        "bin_iou": 0.5344158985024906,
        "bin_mae": 0.11214208473982923,
        "bin_ber": 0.14690697233800676,
        "bin_img_acc": 0.8012103271484374,
        "bin_mir_acc": 0.904825665811351,
        "bin_F1": 0.4584271420251612,
    },
    "FT 50E Freeze": {
        "iou": 0.37243223891510213,
        "mae": 0.5046523481139338,
        "ber": 0.49999810852525495,
        "img_acc": 0.22800418873732003,
        "mir_acc": 0.9999927718119382,
        "F1": 0.6866936317818912,
        "bin_iou": 0.2612560568329271,
        "bin_mae": 0.5685937238925415,
        "bin_ber": 0.3649033668225614,
        "bin_img_acc": 0.22800418873732003,
        "bin_mir_acc": 0.9999927718119382,
        "bin_F1": 0.6866936317818912,
    },
}
